We start loading the required python modules.

In [39]:
import pandas as pd
import numpy as np
from saaty import ahp

And the dummy data for the example.

In [40]:
data = pd.read_excel("data/dummy_survey_data.xlsx")
dummy = pd.read_excel("data/dummy_asneves.xlsx")

As we can see, data is a sample of dummy responses to a saaty questionnaire that evaluates the main attributes env, coste, social and econ; and the subattributes nocons, zepaozec and zepayzedc that are located inside env.

In [41]:
data.head()

,REGISTRO,1_econ_social,1_resp,2_econ_env,2_resp,3_econ_coste,3_resp,4_env_coste,4_resp,5_env_social,5_resp,6_social_coste,6_resp,7_nocons_zepaozec,7_resp,8_nocons_zepayzec,8_resp,9_zepaozec_zepayzec,9_resp
0,45,1,5,1,5,1,6,2,7,2,9,1,5,1,5,1,6,1,5
1,56,2,9,2,9,2,9,1,7,1,9,1,8,2,9,2,9,2,9
2,58,1,6,1,5,2,6,1,5,1,5,2,6,1,5,1,6,2,5
3,59,1,9,1,7,1,5,2,7,1,8,2,7,2,7,2,8,2,7
4,60,1,7,2,9,1,8,1,9,1,9,1,9,2,9,1,8,1,8


We define the squema for the ahp analysis. In this case there are two layers of attributes: main and env.

In [42]:
squema = {
        "main":{
            "range":[1,13],
            "attributes":["econ","social","env","coste"]
        }
    }

The first step is to "train" the model with the survey dummy data: 

In [43]:
res = ahp(
  train=True, # either train the squema from survey data or pass the pretrained squema through the squema argument
  data=data, # Survey data
  pow_value=10, # Pow value to invert the matrix
  confidence=0.95, # Confidence level
  cratio_threshold=1, # CR ratio threshold for filtering responses
  squema=squema # Previously defined squema
)

As a result we can see the weights result for every respondent for every level of the hierarchy:

In [44]:
res.weights['main'].head()

,econ,social,env,coste
0,0.575050,0.270541,0.041410,0.113000
1,0.025368,0.226043,0.666080,0.082510
2,0.603825,0.030860,0.097529,0.267786
3,0.207756,0.082577,0.683096,0.026570
4,0.043110,0.241866,0.623522,0.091503


As well as their consistency ratio

In [46]:
res.cr[0:10]

0    0.330175
1    0.467657
2    0.299182
3    0.434959
4    0.284975
5    0.114592
6    0.956655
7    0.287853
8    0.465639
9    0.379011
Name: cr_ratio, dtype: float64

The average global weights can be seen directly in the squema:

In [47]:
res.squema['main']['weights_mean']

econ      0.210598
social    0.238726
env       0.405357
coste     0.145320
dtype: float64

In a second step we can use those precalculated weights to get a ranking of the followingn dummy alternatives:

In [48]:
dummy.head()

,Alternatives,econ,env,social,coste
0,Zone A,2.673223e+06,10550.000000,12,602638.50
1,Zone B,8.123286e+06,86.980000,100,216993.84
2,Zone C,5.232053e+06,9.739231,50,249695.46


We get the ranking storing it in the df variable and specifying that the attribute coste is a negative attribute (the lower its value the best)

In [49]:
df = res.set_ahp_weights(dummy, inverse = ['coste'])
df

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

,econ,env,social,coste,weight
Alternatives,,,,,
Zone A,0.166779,0.990916,0.074074,0.161532,0.477955
Zone B,0.506801,0.008170,0.617284,0.448610,0.322596
Zone C,0.326421,0.000915,0.308642,0.389857,0.199449


In [50]:
res.attribute_pairwise_matrix

{'econ': Alternatives    Zone A    Zone B    Zone C  priorities
 Alternatives                                          
 Zone A        1.000000  0.329081  0.510932    0.166779
 Zone B        3.038761  1.000000  1.552600    0.506801
 Zone C        1.957208  0.644081  1.000000    0.326421,
 'env': Alternatives    Zone A      Zone B       Zone C  priorities
 Alternatives                                               
 Zone A        1.000000  121.292251  1083.247769    0.990916
 Zone B        0.008245    1.000000     8.930890    0.008170
 Zone C        0.000923    0.111971     1.000000    0.000915,
 'social': Alternatives    Zone A  Zone B  Zone C  priorities
 Alternatives                                      
 Zone A        1.000000    0.12    0.24    0.074074
 Zone B        8.333333    1.00    2.00    0.617284
 Zone C        4.166667    0.50    1.00    0.308642,
 'coste': Alternatives    Zone A    Zone B    Zone C  priorities
 Alternatives                                          
 Zone 